In [ ]:
import sys, os

path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, 'TestBeam')))
print(path2add)

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

In [ ]:
from pathlib import Path
from natsort import natsorted
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')

from beamtest_analysis_helper import toSingleDataFrame_newEventModel_moneyplot

In [ ]:
board_name = "ET2p03"
run_name = "Run_3"

In [ ]:
full_path = '/home/daq/ETROC2/ETROC-Data/2024-07-13_Array_Test_Results'
dirs = natsorted(list(Path(full_path).glob(f'{board_name}_VRef_SCurve_TDC_Pixel_*_QInj_*_Threshold_*')))
dirs[:3]

## Translate nem files to dataframe

In [ ]:
df = toSingleDataFrame_newEventModel_moneyplot(directories=dirs)

In [ ]:
df.to_feather(f'{board_name}_{run_name}_qinj_moneyplot.feather')

In [ ]:
### Drop unnecessary columns
df.drop(columns=['ea', 'bcid', 'l1a_counter'], inplace=True)

## CAL code filtering

In [ ]:
cal_table = df.pivot_table(index=["row", "col", "charge", "threshold"], columns=["board"], values=["cal"], aggfunc=lambda x: x.mode().iat[0])

cal_table = cal_table.reset_index().set_index([('row', ''), ('col', ''), ('charge', ''), ('threshold', '')]).stack().reset_index()
cal_table.columns = ['row', 'col', 'charge', 'threshold', 'board', 'cal_mode']

merged_df = pd.merge(df, cal_table, on=['board', 'row', 'col', 'charge','threshold'])
del df, cal_table

cal_condition = abs(merged_df['cal'] - merged_df['cal_mode']) <= 3
merged_df = merged_df[cal_condition].drop(columns=['cal_mode'])

cal_filtered_df = merged_df.reset_index(drop=True)
cal_filtered_df['board'] = cal_filtered_df['board'].astype(np.uint8)
del merged_df, cal_condition

## TOT code filtering

In [ ]:
tot_table = cal_filtered_df.pivot_table(index=["row", "col", "charge", "threshold"], columns=["board"], values=["tot"], aggfunc=lambda x: x.mode().iat[0])

tot_table = tot_table.reset_index().set_index([('row', ''), ('col', ''), ('charge', ''), ('threshold', '')]).stack().reset_index()
tot_table.columns = ['row', 'col', 'charge', 'threshold', 'board', 'tot_mode']

merged_df = pd.merge(cal_filtered_df, tot_table, on=['board', 'row', 'col', 'charge','threshold'])
del cal_filtered_df, tot_table

tot_condition = abs(merged_df['tot'] - merged_df['tot_mode']) <= 100
merged_df = merged_df[tot_condition].drop(columns=['tot_mode'])

tot_filtered_df = merged_df.reset_index(drop=True)
tot_filtered_df['board'] = tot_filtered_df['board'].astype(np.uint8)
del merged_df, tot_condition

## Calculate mean and std

In [ ]:
grouped = tot_filtered_df.groupby(['row', 'col', 'charge', 'threshold'])
agg_df = grouped.agg(
    cal_mean=('cal', 'mean'),
    cal_std=('cal', 'std'),
    toa_mean = ('toa','mean'),
    toa_std = ('toa', 'std'),
    tot_mean=('tot', 'mean'),
    tot_std=('tot', 'std'),
)
agg_df.to_pickle(f'{board_name}_{run_name}_TDC_Summary.pickle')

## Drawing plots

In [ ]:
row_col_combinations = agg_df.index.to_frame(index=False)[['row', 'col']].drop_duplicates(subset=['row', 'col'])

for (row, col) in row_col_combinations.values:
    subset = agg_df.xs((row, col), level=('row', 'col'))
    charges = subset.index.get_level_values('charge').unique()

    fig, axes = plt.subplots(1, 2, figsize=(22, 10))

    for charge in charges:
        charge_data = subset.xs(charge, level='charge')
        axes[0].plot(charge_data.index.get_level_values('threshold'), charge_data['cal_mean'], '.-', label=f'{charge} fC')
        axes[1].plot(charge_data.index.get_level_values('threshold'), charge_data['cal_std'], '.-', label=f'{charge} fC')

    hep.cms.text(loc=0, ax=axes[0], text="ETL ETROC", fontsize=18)
    hep.cms.text(loc=0, ax=axes[1], text="ETL ETROC", fontsize=18)
    axes[0].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)
    axes[1].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)

    axes[0].set_ylim(charge_data['cal_mean'].mean()-5, charge_data['cal_mean'].mean()+5)
    axes[1].set_ylim(-0.03, 1)

    axes[0].set_xlabel('Threshold')
    axes[1].set_xlabel('Threshold')
    axes[0].set_ylabel('CAL Mean')
    axes[1].set_ylabel('CAL Std')
    axes[0].legend()
    axes[1].legend()

    plt.tight_layout()
    # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_CAL.png')
    # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_CAL.pdf')
    # plt.close(fig)

In [ ]:
row_col_combinations = agg_df.index.to_frame(index=False)[['row', 'col']].drop_duplicates(subset=['row', 'col'])

for (row, col) in row_col_combinations.values:
    subset = agg_df.xs((row, col), level=('row', 'col'))
    charges = subset.index.get_level_values('charge').unique()

    fig, axes = plt.subplots(1, 2, figsize=(22, 10))

    for charge in charges:
        charge_data = subset.xs(charge, level='charge')
        axes[0].plot(charge_data.index.get_level_values('threshold'), charge_data['toa_mean'], '.-', label=f'{charge} fC')
        axes[1].plot(charge_data.index.get_level_values('threshold'), charge_data['toa_std'], '.-', label=f'{charge} fC')

    hep.cms.text(loc=0, ax=axes[0], text="ETL ETROC", fontsize=18)
    hep.cms.text(loc=0, ax=axes[1], text="ETL ETROC", fontsize=18)
    axes[0].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)
    axes[1].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)

    axes[1].set_ylim(-0.03, 4)
    axes[0].set_ylim(200, 290)

    axes[0].set_xlabel('Threshold')
    axes[1].set_xlabel('Threshold')
    axes[0].set_ylabel('TOA Mean')
    axes[1].set_ylabel('TOA Std')
    axes[0].legend()
    axes[1].legend()

    plt.tight_layout()
    # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_TOA.png')
    # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_TOA.pdf')
    # plt.close(fig)

In [ ]:
row_col_combinations = agg_df.index.to_frame(index=False)[['row', 'col']].drop_duplicates(subset=['row', 'col'])

for (row, col) in row_col_combinations.values:
    subset = agg_df.xs((row, col), level=('row', 'col'))
    charges = subset.index.get_level_values('charge').unique()

    fig, axes = plt.subplots(1, 2, figsize=(22, 10))

    for charge in charges:
        charge_data = subset.xs(charge, level='charge')
        axes[0].plot(charge_data.index.get_level_values('threshold'), charge_data['tot_mean'], '.-', label=f'{charge} fC')
        axes[1].plot(charge_data.index.get_level_values('threshold'), charge_data['tot_std'], '.-', label=f'{charge} fC')

    hep.cms.text(loc=0, ax=axes[0], text="ETL ETROC", fontsize=18)
    hep.cms.text(loc=0, ax=axes[1], text="ETL ETROC", fontsize=18)
    axes[0].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)
    axes[1].set_title(f'Row {row}, Col {col}', loc='right', fontsize=16)

    axes[1].set_ylim(-0.03, 4)

    axes[0].set_xlabel('Threshold')
    axes[1].set_xlabel('Threshold')
    axes[0].set_ylabel('TOT Mean')
    axes[1].set_ylabel('TOT Std')
    axes[0].legend()
    axes[1].legend()

    plt.tight_layout()
    # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_TOT.png')
    # fig.savefig(f'{board_name}/{board_name}_{run_name}_Row_{row}_Col_{col}_TOT.pdf')
    # plt.close(fig)